## Добавляем данные о числе жильцов в таблицу с данными по теплоучету

In [1]:
import pandas as pd
import numpy as np

import difflib
import re

from tqdm import tqdm

### Загружаем таблицу с данными по числу жильцов, извлекаем из нее все адреса

In [2]:
inhabs_data = pd.read_csv('inhabitants.csv', sep=';', encoding='cp1251')
new_add = inhabs_data['Address']
new_add

0                      г. Томск, пер. 1905 года, д. 8
1        обл. Томская, г. Томск, пер. 1905 года, д. 8
2                     г. Томск, пер. 1905 года, д. 15
3       обл. Томская, г. Томск, пер. 1905 года, д. 15
4                     г. Томск, пер. 1905 года, д. 20
                            ...                      
6672      обл. Томская, г. Томск, ул. Яковлева, д. 45
6673                    г. Томск, ул. Яковлева, д. 55
6674      обл. Томская, г. Томск, ул. Яковлева, д. 55
6675                  г. Томск, ул. Яковлева, д. 74/1
6676    обл. Томская, г. Томск, ул. Яковлева, д. 74/1
Name: Address, Length: 6677, dtype: object

### Загружаем таблицу с данными по теплоучету, также извлекаем из нее адреса

In [3]:
data = pd.read_csv('dataset_3_part_address.csv', sep=';', parse_dates=['Date'], decimal='.')
old_add = data['Address'].drop_duplicates()
old_add

0           Иркутский тр. , д.196
1         Иркутский тр. , д.104-а
2           Иркутский тр. , д.108
3             Фрунзе пр. , д.67/1
4           Иркутский тр. , д.114
                   ...           
470988        Нахимова ул. , д.12
565652     Профсоюзная ул. , д.20
571161        Косарева ул. , д.14
573882      Котовского ул. , д.12
696386          Кирова пр. , д.22
Name: Address, Length: 1093, dtype: object

In [4]:
old_list = list(old_add)
new_list = list(new_add)

### Модифицируем формат адресов в датасете по теплоучету, чтобы он больше соответствовал новым данным, при этом организуем словарь "новое значение" - "старое значение" для того, чтобы можно было вернуться к исходным значениям

In [5]:
old_list_mod = []
old_mod_dict = dict()
for elem in old_list:
    copy = elem
    copy = re.sub('С.Лазо', 'Сергея Лазо', copy, count=1)
    if (re.match('.*пер.', copy)):
        copy = 'пер. ' + re.sub(' пер. ', '', copy)
    if (re.match('.*ул.', copy)):
        copy = 'ул. ' + re.sub(' ул. ', '', copy)
    if (re.match('.*пр-д', copy)):
        copy = 'проезд. ' + re.sub(' пр-д ', '', copy)
    old_list_mod.append(copy)
    old_mod_dict.update({copy:elem})

### Также производим некоторые операции с форматом адресов из таблицы с данными по числу жильцов

In [6]:
new_list_mod = []
new_mod_dict = dict()
for elem in new_list:
    copy = elem
    copy = re.sub('МПС Северный', 'МПС', copy, count=1)
    if re.match('обл. Томская, г. Томск, ', copy):
        new_list_mod.append(copy[24:])
        new_mod_dict.update({copy[24:]:elem})
    elif re.match('г. Томск, ', copy):
        new_list_mod.append(copy[10:])
        new_mod_dict.update({copy[10:]:elem})

### Для каждого модифицированного адреса из таблицы с данными по теплоучету подбираем модифицированный адрес из таблицы с данными по числу жильцов, параллельно заполняя словарь соответствий. В словарь при этом записываются не модифицированные, а "старые" адреса из обоих таблиц, получаемые из двух соответствующих словарей, созданных ранее

In [7]:
add_match_d = dict()
for address in tqdm(old_list_mod):
    match = difflib.get_close_matches(''.join(address.split()), new_list_mod, n=1, cutoff=0)[0]
    add_match_d.update({old_mod_dict[address]:new_mod_dict[match]})

100%|██████████████████████████████████████████████████████████████████████████████| 1093/1093 [18:05<00:00,  1.01it/s]


In [8]:
match_df = pd.DataFrame.from_dict(add_match_d, orient='index')
match_df.reset_index(inplace=True)
match_df.columns = ['old', 'new']

In [9]:
match_df.head()

,old,new
0,"Иркутский тр. , д.196","г. Томск, тракт Иркутский, д. 196"
1,"Иркутский тр. , д.104-а","г. Томск, тракт Иркутский, д. 104а"
2,"Иркутский тр. , д.108","г. Томск, тракт Иркутский, д. 108"
3,"Фрунзе пр. , д.67/1","г. Томск, пр-кт Фрунзе, д. 67/1"
4,"Иркутский тр. , д.114","г. Томск, тракт Иркутский, д. 114"


In [10]:
match_df.to_clipboard(index=False)

### Получив таблицу соответствий, проверяем ее корректность вручную

#### Для некоторых адресов подобрать соответствие не получилось даже вручную ввиду отстутствия данных на сайте ЖКХ

In [11]:
add_match_d.update({'Беринга ул. , д.2/2':'None'})
add_match_d.update({'Мичурина ул. , д.55':'None'})
add_match_d.update({'Интернационалистов ул. , д.21':'None'})
add_match_d.update({'Осенний пер. , д.2':'None'})
add_match_d.update({'Мичурина ул. , д.51':'None'})
add_match_d.update({'Угрюмова Ал. ул. , д.1/1':'None'})

#### Для остальных адресов можно вручную устранить ошибки при автоматическом поиске соответствия (данные по ним присутствовали)

In [12]:
add_match_d.update({'Дзержинского ул. , д.24-a':'г. Томск, ул. Дзержинского, д. 24а'})
add_match_d.update({'Рабочая 3 ул. , д.9':'г. Томск, ул. Рабочая 3-я, д. 9'})
add_match_d.update({'Рабочая 1 ул. , д.8':'г. Томск, ул. Рабочая 1-я, д. 8'})

### После того как убедились в корректности всех соответствий, на их основе можно внести в таблицу данные по числу жильцов.

In [13]:
data['New_Address'] = data['Address'].map(add_match_d)

In [14]:
new_add_to_num_dict = {}
for add,num in zip(list(inhabs_data['Address']), list(inhabs_data['Inhabitants'])):
    new_add_to_num_dict.update({add:num})

In [15]:
data['Inhabitants'] = data['New_Address'].map(new_add_to_num_dict)
data = data.drop(['Address', 'New_Address'], axis=1)
data.head()

,Date,М1_t,М2_t,delta_М_t,Т1,Т2,delta_Т,Q,USPD,YYYYMM,registrated,scheme,type,area,floors,walls_material,year_of_construction,area_of_building,temp,Inhabitants
0,2013-12-01,93.950333,84.168098,9.782235,65.859886,43.560223,22.299663,2.104783,1,201312,Heating + Hot water,opened,1105,2707.4,5,panel,1983,4401.0,-0.157917,102.0
1,2013-12-01,169.336258,169.643768,-0.307510,66.641441,49.602718,17.038723,2.892227,3,201312,Heating + Hot water,closed,2105,3358.4,5,brick,1974,3610.4,-0.157917,133.0
2,2013-12-01,98.604866,100.520248,-1.915382,67.661072,41.020565,26.640507,2.630709,4,201312,Heating + Hot water,closed,2105,3153.5,5,brick,1975,3150.2,-0.157917,119.0
3,2013-12-01,121.030000,115.590000,5.440000,66.470000,50.350000,16.120000,1.949000,10,201312,Heating + Hot water,opened,1105,2646.5,5,panel,1982,2932.7,-0.157917,100.0
4,2013-12-01,100.120705,102.018661,-1.897956,67.699661,40.780910,26.918751,2.698659,12,201312,Heating + Hot water,closed,2105,3187.5,5,brick,1976,3456.2,-0.157917,132.0


### Сохраняем таблицу с обновленными данными в csv

In [16]:
data.to_csv('dataset_4.csv', sep=';', index=False, encoding='cp1251')

### Добавляем столбец с числом жильцов в таблицу с очищенными данными без адресов на основе соответствий УСПД-число_жильцов из обновленной таблицы.

In [17]:
data_cleared = pd.read_csv('dataset_3_part.csv', sep=';')

In [18]:
uspd_inhabs_dict = {}
for uspd,num in zip(list(data['USPD']), list(data['Inhabitants'])):
    uspd_inhabs_dict.update({uspd:num})

In [19]:
data_cleared['Inhabitants'] = data_cleared['USPD'].map(uspd_inhabs_dict)

In [20]:
data_cleared.to_csv('dataset_4_cleared.csv', sep=';', index=False, encoding='cp1251')